# Experiments with the baseline metric

Kísérlet, ha változtatjuk a timeframe-t, akkor hogyan változik a baseline mérték NDCG-je

In [1]:
import sys
sys.path.insert(0,'./../script')
import linkpred_eval
import test as t

import pandas as pd
import math
import json
import numpy as np

In [2]:
%matplotlib inline

In [3]:
# a config fájlban szereplő paramétereket itt is beolvassuk
with open("/home/danielolah/git/pyRecSys/python/experiments/config_test.json") as config_file:    
    parameters = json.load(config_file)
    
# az eredeti adathalmaz beolvasása
# ennek kellett egy másik változó, ami a 3 oszlopos (time, source, target) adathalmazra mutat
filepath = parameters["input_file_with3column"]
edges_df = pd.read_csv(filepath, sep=' ', names=["time","source","target"])

# készít egy új oszlopot, hogy az adott élek hanyadik napon (periódusban) jelentek meg
timeFrame = parameters["timeFrame"]
minTime = parameters["minTime"]
edges_df['day'] = edges_df['time'].map(lambda x: math.floor((x-minTime)/timeFrame))

In [19]:
# temp pr számolás előkészületek
G = t.get_graph_from_df(edges_df)
nodes = G.nodes()
temp_pr_scores = np.outer(nodes, [1,0.0,0.0,0.0,0.0]) # columns: node label, temp pr score, active mass
nodes_array = np.asarray(nodes)

# temporal pagerank jumping and transition probability
alpha = 0.85
beta = 0.9

NDCG_averages = []

# 24ó, 12ó, 6ó, 3ó, 2ó, 1ó
for timeFrame in [86400, 43200, 21600, 10800, 7200, 3600]:
    edges_df['day'] = edges_df['time'].map(lambda x: math.floor((x-minTime)/timeFrame))
    ndays = len(edges_df['day'].unique())
    NDCG_values = np.zeros((ndays,3)) # plusz egy oszlop: hány él jelent meg az egyes intervallumokban
    NDCG_k = 100
    for day in range(ndays+1):
        edges_real = edges_df[edges_df['day'] == day]
        # valós élek egy dataframe-ben

        # 3. és 4. oszlopba átmásolni az aktuális értékeket az 1. és 2. oszlopból
        temp_pr_scores[:,3:5] = temp_pr_scores[:,1:3]

        # 1. és 2. oszlopban frissíteni a valódiak alapján
        for i, edge in edges_real.iterrows():
            # get the row index of the source and target node
            edge_target_index = nodes_array.searchsorted(edge[2])
            edge_source_index = nodes_array.searchsorted(edge[1])

            # temporal pagerank
            temp_pr_scores[edge_source_index,1] += 1-alpha
            temp_pr_scores[edge_source_index,2] += 1-alpha
            temp_pr_scores[edge_target_index,1] += temp_pr_scores[edge_source_index,2]*alpha
            temp_pr_scores[edge_target_index,2] += temp_pr_scores[edge_source_index,2]*beta*alpha
            temp_pr_scores[edge_source_index,2] *= (1-beta)

        # fájlba kiírni a PageRank értékeket
        # np.savetxt("./../data/PRscores_day%i" % day, temp_pr_scores, delimiter="|")

        # összehasonljuk az aktuálisat az eggyel korábbival
        if day>0:
            # baseline DCG value
            temp_pr_scores_sorted = temp_pr_scores[temp_pr_scores[:,3].argsort()[::-1]]
            DCG = 0.0
            for i in range(NDCG_k):
                DCG += temp_pr_scores_sorted[i,1] / math.log2(i+2)

            # normalization
            temp_pr_scores_sorted = temp_pr_scores[temp_pr_scores[:,1].argsort()[::-1]]
            IDCG = 0.0
            for i in range(NDCG_k):
                IDCG += temp_pr_scores_sorted[i,1] / math.log2(i+2)

            NDCG_values[day-1,:] = [day, DCG / IDCG, len(edges_real)]
            
    
    NDCG_averages.append(np.average(NDCG_values[:,1], weights=NDCG_values[:,2]))

In [16]:
NDCG_averages

[0.97796811252329041,
 0.99849191054628872,
 0.99971695958042261,
 0.99993701726800521,
 0.99997161761697151,
 0.99998860411828017]

In [20]:
NDCG_averages

[0.90253887750950268,
 0.99098384793519667,
 0.99797974758006414,
 0.99960999093816361,
 0.99980336259686498,
 0.99990865334203927]

In [17]:
a = np.array([[1,2],[2,0],[3,1]])
a

array([[1, 2],
       [2, 0],
       [3, 1]])

In [18]:
np.average(a[:,0], weights=a[:,1])

1.6666666666666667